# PubMed API Search

This script performs a pubmed search and gathers the pubmed IDs of papers that meet all of our concept, affiliation, author, and other criteria.

Uses the Entrez E-utilities: 

https://www.ncbi.nlm.nih.gov/books/NBK25497/#chapter2.Introduction

https://pubmed.ncbi.nlm.nih.gov/help/

Tags:
https://www.ncbi.nlm.nih.gov/pmc/about/userguide/

In [11]:
from bs4 import BeautifulSoup
import requests

## 1) Tidy up inputs

### Date (changes quarterly)

In [2]:
# pubdate = '2023/07/01:2023/09/30[pdat]'

pubdate = '2023/03/01:2023/06/30[pdat]'

### Author list 

Was split by author speciality and had redundancies. It seems that " ` " and " ' " are interchangable, and that a space in front of the name doesn't matter.

Manually replace:
- ' with `
- `” [Au] “` with `” [Au] OR “`
- `[Au]"` with `"[Au]`

In [3]:
# Copy/pasted from word doc
authors_clinic = '“Nag D”[Au] OR “Duffy F”[Au] OR “Kearney J”[Au] OR “Trehan I”[Au] OR “Cangelosi G”[Au] OR “Andriesen J”[Au] OR “Coler R”[Au] OR “Kgoadi K”[Au] OR “Fiore-Gartland  A”[Au] OR “Johnson A”[Au] OR “Rais M”[Au] OR “Sanders L”[Au] OR “Attia E”[Au] or “Horne D”[Au] OR “Okwaro E”[Au] OR “Tapley A”[Au] OR “Stein G”[Au] OR “Sorri Y”[Au] OR “Bender Ignacio R”[Au] OR “Wald A”[Au] OR “Jaspan H”[Au] OR “Kumar A”[Au] OR “Pecor T”[Au] OR “Zhu Z”[Au] OR “Mudrock E”[Au] OR “Olson A”[Au] OR “Kublin J”[Au] OR “Randhawa A”[Au] OR “Andersen-N E”[Au] OR “Mayer-Blackwell K”[Au] OR “Kieswetter N”[Au] OR “Harband M”[Au] OR “Harne R”[Au] OR “Akins S”[Au] OR “Larsen Akins S”[Au] OR “Valerie R”[Au] OR “Gottlieb G”[Au] OR “Shapiro A”[Au] OR “LaCourse S”[Au] OR “Graham S”[Au] OR “Smytheman T”[Au] OR “Cowan J”[Au] OR “Iribarren S”[Au]'
authors_compbio = '“Nag D” [Au] OR “Duffy F” [Au] OR “Fiore-Gartland A” [Au] OR “Mayer-Blackwell, K” [Au] OR “Kieswetter N” [Au] OR “Abdelaal H” [Au] OR “Segnitz M” [Au] OR “Ma S” [Au] OR “Pepper E” [Au] OR “Allen R” [Au] OR “Mixon T” [Au] OR “Himmelfarb T” [Au] OR “Bishop E” [Au] OR “Bustad E” [Au] OR “Grosvenor D” [Au]'
authors_epi = '“Nag D” [Au] OR “Kearney J” [Au] OR “Trehan I” [Au] OR “Cangelosi G” [Au] or “Johnson A” [Au] or “Johnson A” [Au] OR “Rais M” [Au] or “Attia E” [Au] OR “Horne D” [Au] OR “Okwaro E” [Au] OR “Sorri Y” [Au] OR “Bender Ignacio R” [Au] OR “Wald A” [Au] OR “Kublin J” [Au] OR “Gottlieb G” [Au] OR “Shapiro A” [Au] OR “LaCourse S” [Au] OR “Graham S” [Au] OR “Motiri F” [Au] OR “LeGrand K” [Au] OR “Cherkos A” [Au] OR “Fajans M” [Au] OR “Tram K” [Au] OR “Basting A” [Au] OR “Ross J” [Au] OR “King`ori B” [Au] OR “Ndoti A” [Au] OR “Black D” [Au] OR “Magomere R” [Au] OR “Kerani R” [Au] OR “Church E” [Au] OR  “Escudero J” [Au] OR “John-Stewart G” [Au] OR “Dalmat R” [Au] OR “Berkoh H” [Au]'
authors_globalhealth = '“Nag D” [Au] OR “Kearney J” [Au] OR “Trehan I” [Au] OR “Cangelosi G” [Au] or “Johnson A” [Au] or “Johnson A” [Au] OR “Rais M” [Au] or “Attia E” [Au] OR “Horne D” [Au] OR “Okwaro E” [Au] OR “Kublin J” [Au] OR “Gottlieb G” [Au] OR “Shapiro A” [Au] OR “LaCourse S” [Au] OR “Graham S” [Au] OR “Motiri F” [Au] OR “LeGrand K” [Au] OR “Cherkos A” [Au] OR “Fajans M” [Au] OR “Tram K” [Au] OR “Basting A” [Au] OR “Ndoti A” [Au] OR “Black D” [Au] OR “Escudero J” [Au] OR “John-Stewart G” [Au] OR “Abdelaal H” [Au] OR “Coler R” [Au] OR “Sanders L” [Au] OR “Tapley A” [Au] OR “Stein G” [Au] OR “Jaspan H” [Au] OR “Kumar A” [Au] OR “Pecor T” [Au] OR “Randhawa A” [Au] OR “Andersen-Nissen E” [Au] OR “Smytheman T” [Au] OR “Cowan J” [Au] OR “Iribarren S” [Au] OR “Sharma M” [Au] OR “Henry N” [Au] OR “Brumwell A” [Au] OR “Buckner F” [Au] OR “Duncombe C” [Au] OR “Armistead B” [Au] OR “Panpradist N” [Au] OR “Hill D” [Au] OR “Miller D” [Au] OR “Haynes A” [Au] OR “Chohan B” [Au] OR “Seshadri C” [Au] OR “Williams B” [Au] OR “Barrett H” [Au] OR “West E” [Au]'
authors_immun = '“Nag D” [Au] OR “Kearney J” [Au] OR “Johnson A” [Au] OR “Rais M” [Au] or “Kublin J” [Au] OR “Ndoti A” [Au] OR “Abdelaal H” [Au] OR “Coler R” [Au] OR “Jaspan H” [Au] OR “Kumar A” [Au] OR “Pecor T” [Au] OR “Randhawa A” [Au] OR “Andersen-Nissen E” [Au] OR “Duncombe C” [Au] OR “Armistead B” [Au] OR “Miller D” [Au] OR “Seshadri C” [Au] OR “Williams B” [Au] OR “Barrett H” [Au] OR “West E” [Au] OR “Church E” [Au] OR  “Duffy F” [Au] OR “Fiore-Gartland A” [Au] OR “Mayer-Blackwell K” [Au] OR “Kieswetter N” [Au] OR “Allen R” [Au] OR “Himmelfarb T” [Au] OR “Bishop E” [Au] OR “Kgoadi K” [Au] OR “Zhu Z” [Au] OR “Harband M” [Au] OR “Harne R” [Au] OR “Akins S” [Au] OR “Larsen Akins S” [Au] OR “Reese V” [Au] OR “Pepple K” [Au] OR “Chendi B” [Au] OR “Subramanian N” [Au] OR “Bhuiyan M” [Au] OR “Kaushansky A” [Au] OR “Anterasian C” [Au] OR “Maciag K” [Au] OR “Fleming L” [Au] OR “Huynh T H” [Au] OR “Harrington W” [Au] OR “Harding C” [Au] OR “Kim H” [Au] OR “Ho K” [Au] OR “Klas J” [Au] OR “Gasper M" [Au] OR “Simmons J” [Au] OR “Fernandez M” [Au] OR “Krug S” [Au] OR “Le C” [Au] OR “Weigel K” [Au] OR “Gern B” [Au] OR “Cross L” [Au] OR “Urdahl K” [Au] OR “Plumlee C” [Au] OR “Cohen S” [Au] OR “Chen S” [Au] OR “Baldwin S” [Au] OR “Gerner M” [Au] OR “Shah J” [Au] OR “Reynolds A” [Au] OR “Winter C” [Au] OR “Shamskhou E” [Au] OR “ Foster K” [Au] OR “Maerz M” [Au] OR “Villagrana P” [Au] OR “Tappen V” [Au] OR “Files M” [Au] OR “Makatsa M” [Au] OR “Hawn T” [Au] OR “Koelle D” [Au] OR “ Ramirez E” [Au] OR “Layton E” [Au] OR “Phan J” [Au] OR  “Yu K” [Au] OR “ Richardson S” [Au]'
authors_micro = '“Nag D” [Au] OR “Kearney J” [Au] OR “Johnson A” [Au] OR “Rais M” [Au] OR “Abdelaal H” [Au] OR “Coler R” [Au] OR “Jaspan H” [Au] OR “Kumar A” [Au] OR “Pecor T” [Au] OR “Armistead B” [Au] OR “Miller D” [Au] OR “Allen R” [Au] OR “Kgoadi K” [Au] OR “Zhu Z” [Au] OR “Subramanian N” [Au] OR “Bhuiyan M” [Au] OR “Kaushansky A” [Au] OR “Fleming L” [Au] OR “Huynh T H” [Au] OR “Harrington W” [Au] OR “Harding C” [Au] OR “Kim H” [Au] OR “Klas J” [Au] OR “ Gasper M” [Au] OR “Simmons J” [Au] OR “Fernandez M” [Au] OR “Krug S” [Au] OR “Le C” [Au] OR “Weigel K” [Au] Or “Cangelosi G” [Au] OR “Sanders L” [Au] OR “Buckner F” [Au] OR “Panpradist N” [Au] OR “Haynes A” [Au] OR “Chohan B” [Au] OR “King`ori B” [Au] OR “Ma S” [Au] OR “Pepper E” [Au] OR “Mixon T” [Au] OR “Olson A” [Au] OR “Galina L” [Au] OR “Boradia V” [Au] OR “Parish T” [Au] OR “Chowdhury S” [Au] OR “Peterson E” [Au] OR “Sherman D” [Au] OR “Hernandez R” [Au] OR “Tieu E” [Au] OR “Deshpande A” [Au] OR “Bhagwat A” [Au] OR “Frando A” [Au] OR “Rodrigues da Costa F” [Au] OR “Eydinova A” [Au] OR “Butts A” [Au] OR “Berube B” [Au] OR “Ames L” [Au] OR “Coldren M” [Au] OR “Lial R” [Au] OR “Jones M” [Au] OR “Grundner C” [Au] OR “Carroll B” [Au] OR “Lamot E” [Au] OR “Fredricks L” [Au] OR “Eldesouky H” [Au] OR “Brache J” [Au] OR “Adams K” [Au] OR “Pruneda A” [Au] OR “Nilles E” [Au] OR “Wier J” [Au] OR “Thong Q” [Au]'
authors_modeling = '“Nag D” [Au] OR “Kearney J” [Au] OR “Abdelaal H” [Au] OR “Duffy F” [Au] OR “Pepple K” [Au] OR “Motiri F” [Au] OR “LeGrand K” [Au] OR “Cherkos A” [Au] OR “Fajans M” [Au] OR “Tram K” [Au] OR “Basting A” [Au] OR “Sharma M” [Au] OR “Henry N” [Au] OR “Ross J” [Au]'
authors_pubhealth = '“Nag D” [Au] OR “Kearney J” [Au] OR “Motiri F” [Au] OR “LeGrand K” [Au] OR “Cherkos A” [Au] OR “Fajans M” [Au] OR “Tram K” [Au] OR “Basting A” [Au] OR “Johnson A” [Au] OR “Rais M” [Au] OR “Armistead B” [Au] OR “Sanders L” [Au] OR “Panpradist N” [Au] OR “ King`ori B” [Au] OR “Ndoti A” [Au] OR “Trehan I” [Au] OR “Attia E” [Au] OR “Horne D” [Au] OR “Okwaro E” [Au] OR “Black D” [Au] OR “Tapley A” [Au] OR “Stein G” [Au] OR “Brumwell A” [Au] OR “Hill D” [Au] OR “Sorri Y” [Au] OR “Bender Ignacio R” [Au] OR “Wald A” [Au] OR “Magomere R” [Au] OR “Kerani R” [Au] OR “Ranade D” [Au] OR “Segura P” [Au] OR “Budd K” [Au] OR “Ghassemieh B” [Au] OR “Wood R” [Au]'
authors_sysbio = '“Nag D” [Au] OR “Abdelaal H” [Au] OR “Duffy F” [Au] OR “Coler R” [Au] OR “Kgoadi K” [Au] OR “Subramanian N” [Au] OR “Bhuiyan M” [Au] OR “Kaushansky A” [Au] OR “Ma S” [Au] OR “Pepper E” [Au] OR “Mortiz R” [Au] OR “Peterson E” [Au] OR “Sherman D” [Au] OR “Duncombe C” [Au] OR “Fiore-Gartland A” [Au] OR “Anterasian C” [Au] OR “Maciag K” [Au] OR “Ong S” [Au]'
authors_other = '"Buckner FS"[AU] OR "Buckner Fred*"[AU] OR “Mortiz R” [Au] OR “Trehan I” [Au] OR “Brumwell A” [Au] OR “Ranade D” [Au] OR “Pepple K” [Au] OR “Sharma M” [Au] OR “Cangelosi G” [Au] OR “Galina L” [Au] OR “Boradia V” [Au] OR “Parish T” [Au] OR “Chowdhury S” [Au] OR “Chendi B” [Au] OR “Segnitz M” [Au] OR “Andriesen J” [Au]'

Tidy up more

In [4]:
# Combine into one long string, then split into a list of names
authors = ' OR '.join([authors_clinic, authors_compbio, authors_epi, authors_globalhealth,
            authors_immun, authors_micro, authors_modeling, authors_pubhealth,
            authors_sysbio, authors_other]).\
                replace(' or ', ' OR ').\
                replace(' Or ', ' OR ').\
                replace('AU', 'au').\
                replace('Au', 'au').\
                replace('\" [au] \"', '[au] OR ').\
                split('[au] OR ')

# Remove whitespace
author_names = [s.strip() for s in authors]  # leading and trailing

# Handle special (last) case
author_names2 = [s.replace('“Andriesen J” [au]', '“Andriesen J”') for s in author_names] 

# Remove duplicates
author_names3 = list(set(author_names2))

print(len(author_names3))
print(author_names3)

175
['“Panpradist N”', '“Graham S”', '“Rais M”', '“ Ramirez E”', '“Subramanian N”', '“Black D”', '“Eldesouky H”', '“Fiore-Gartland  A”', '“Akins S”', '“Seshadri C”', '“Mixon T”', '“Fredricks L”', '“Horne D”', '“Segnitz M”', '“Klas J”', '“Fernandez M”', '“ Gasper M”', '“Randhawa A”', '“Harding C”', '“Hill D”', '“Pepper E”', '“Andriesen J”', '“Miller D”', '“Baldwin S”', '“Winter C”', '“Segura P”', '“Bustad E”', '“Ames L”', '“Kublin J”', '“Shamskhou E”', '“Olson A”', '“Smytheman T”', '“ Foster K”', '“Escudero J”', '“Peterson E”', '“Fleming L”', '“Cross L”', '“Magomere R”', '“Le C”', '“Kgoadi K”', '“Chendi B”', '“Cowan J”', '“Dalmat R”', '“Duncombe C”', '“Gern B”', '“Bishop E”', '“Bhuiyan M”', '“Mayer-Blackwell K”', '“Bhagwat A”', '“Sharma M”', '“Rodrigues da Costa F”', '“Gottlieb G”', '“Nilles E”', '“Andersen-N E”', '“Maerz M”', '“Berube B”', '“Thong Q”', '“LaCourse S”', '“Adams K”', '“Files M”', '“Lial R”', '“Carroll B”', '“Yu K”', '“Gerner M”', '“King`ori B”', '“Maciag K”', '“Armistead 

Split into two big strings. 90 authors is ok but 99+ and it the URL request doesn't return anything.

In [6]:
authors_str1 = '[au] OR '.join(author_names3[:90])
authors_final1 = '(' + authors_str1 + ')'

authors_str2 = '[au] OR '.join(author_names3[90:])
authors_final2 = '(' + authors_str2 + ')'

### Other specifications

Don't change regularly. Copy/pasted from "TB Center Metrics Analysis - revised April 2023.docx". 

In [7]:
concept = '("tubercul*"[tw] OR "Antitubercul*"[tw] OR "Anti-Tubercul*"[tw] OR "osteotubercul*"[tw] OR "nephrotubercul*"[tw] OR "anthracosilicotubercul*"[tw] OR "coniotubercul*"[tw] OR "Tuberculin"[tw] OR "tb"[tw] OR "xdr-tb"[tw] OR "xdrtb"[tw] OR "mdr-tb"[tw] OR "mdrtb"[tw] OR "phthisis"[tw] OR "pneumonophthisis"[tw] OR "pneumophthisiology"[tw] OR "silicotubercul*"[tw] OR "bazin disease"[tw] OR "erythema induratum"[tw] OR "white swelling"[tw] OR "king`s evil"[tw] OR "scrofula"[tw] OR "pott disease"[tw] OR "koch`s disease"[tw] OR "Interferon-gamma Release Test"[tw] OR "Tuberculosis"[Mesh] OR "Mycobacterium tuberculosis"[Mesh] OR "Antitubercular Agents"[Mesh] OR "Tuberculin Test"[Mesh] OR "Interferon-gamma Release Tests"[Mesh] OR "Tuberculosis Vaccines"[Mesh])'
concept_final = concept.replace('[Mesh]', '[mh]')

affiliation = '("university of washington"[ad] OR "univ washington"[ad] OR "washington.edu"[ad] OR "uw.edu"[ad] OR "fred hutch*"[ad] OR "fred hutchinson"[ad] OR "fredhutch.org"[ad] OR "Tuberculosis Research and Training Center"[ad] OR "Center for Infectious Disease Research"[ad] OR "Infectious Diseases Research Institute"[ad] OR "Infectious Disease Research Institute"[ad] OR "Fhcrc.org"[ad] OR "path"[ad] OR "Veterans Affairs Puget Sound"[ad] OR "VA Puget Sound"[ad] OR "Center for Global Infectious Disease Research"[ad] OR "seattlechild*"[ad] OR "seattlechildrens.org"[ad] OR "Seattle child*"[ad] OR "cidresearch.org"[ad] OR "sbri.org"[ad] OR "bill and melinda"[ad] OR "bill melinda"[ad] OR "gates found*"[ad] OR "gatesfoundation.org"[ad] OR "Harborview"[ad] OR "UW Medical Center"[ad] OR "Seattle Cancer Care Alliance"[ad] OR "TB Discovery Research"[ad] OR "seattle"[ad] OR "puget sound"[ad] OR "seattlebiomed.org"[ad] OR "Seattle biomedical research"[ad] OR "E-Science Institute"[ad] OR "Institute for Systems Biology"[ad] OR "sytemsbiology.org"[ad] OR "Lawrence Berkeley National Laboratory"[ad] OR "isbscience.org"[ad] OR "Seattle Structural Genomics Center"[ad] OR "PAI Life Sciences"[ad] OR "King County"[ad] OR "Kwazulu"[ad] OR "Natal"[ad] OR "Edendale Hospital"[ad] OR "south Africa"[ad] OR "University of Nairobi"[ad] OR "Kenya"[ad] OR "treatmentactiongroup.org"[ad] OR "TAG"[ad] OR "treatment action group"[ad]  OR ("Kenya Medical Research Institute"[ad] OR Hospital"[ad])'

# 2) Build the URLs

When combining all the terms the URL is too large

In [8]:
base = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=' + pubdate

#### Authors


In [174]:
url_authors1 = base +  authors_final1 + '+AND+' + affiliation + '&usehistory=y'
url_authors2 = base +  authors_final2 + '&usehistory=y'

url_authors1

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=2023/07/01:2023/09/30[pdat](“Mortiz R”[au] OR “Attia E”[au] OR “Kaushansky A”[au] OR “Hawn T”[au] OR “Sorri Y”[au] OR “Duffy F”[au] OR “Tapley A”[au] OR “Boradia V”[au] OR “Horne D”[au] OR “Harband M”)+AND+("university of washington"[ad] OR "univ washington"[ad] OR "washington.edu"[ad] OR "uw.edu"[ad] OR "fred hutch*"[ad] OR "fred hutchinson"[ad] OR "fredhutch.org"[ad] OR "Tuberculosis Research and Training Center"[ad] OR "Center for Infectious Disease Research"[ad] OR "Infectious Diseases Research Institute"[ad] OR "Infectious Disease Research Institute"[ad] OR "Fhcrc.org"[ad] OR "path"[ad] OR "Veterans Affairs Puget Sound"[ad] OR "VA Puget Sound"[ad] OR "Center for Global Infectious Disease Research"[ad] OR "seattlechild*"[ad] OR "seattlechildrens.org"[ad] OR "Seattle child*"[ad] OR "cidresearch.org"[ad] OR "sbri.org"[ad] OR "bill and melinda"[ad] OR "bill melinda"[ad] OR "gates found*"[ad] OR "gatesfoundation

#### Concept

In [177]:
url_concept = base + concept + '+AND+' + affiliation + '&usehistory=y'
url_concept

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=2023/07/01:2023/09/30[pdat]("tubercul*"[tw] OR "Antitubercul*"[tw] OR "Anti-Tubercul*"[tw] OR "osteotubercul*"[tw] OR "nephrotubercul*"[tw] OR "anthracosilicotubercul*"[tw] OR "coniotubercul*"[tw] OR "Tuberculin"[tw] OR "tb"[tw] OR "xdr-tb"[tw] OR "xdrtb"[tw] OR "mdr-tb"[tw] OR "mdrtb"[tw] OR "phthisis"[tw] OR "pneumonophthisis"[tw] OR "pneumophthisiology"[tw] OR "silicotubercul*"[tw] OR "bazin disease"[tw] OR "erythema induratum"[tw] OR "white swelling"[tw] OR "king`s evil"[tw] OR "scrofula"[tw] OR "pott disease"[tw] OR "koch`s disease"[tw] OR "Interferon-gamma Release Test"[tw] OR "Tuberculosis"[Mesh] OR "Mycobacterium tuberculosis"[Mesh] OR "Antitubercular Agents"[Mesh] OR "Tuberculin Test"[Mesh] OR "Interferon-gamma Release Tests"[Mesh] OR "Tuberculosis Vaccines"[Mesh])+AND+("university of washington"[ad] OR "univ washington"[ad] OR "washington.edu"[ad] OR "uw.edu"[ad] OR "fred hutch*"[ad] OR "fred hutchinso

#### Affiliation

In [195]:
url_affil = base + authors_str2 + '+AND+' + affiliation + '&usehistory=y'
url_affil

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=2023[pdat]“Cowan J”[au] OR “Basting A”[au] OR “Kieswetter N”[au] OR “Kim H”[au] OR “Kumar A”[au] OR “Chowdhury S”[au] OR “Ghassemieh B”[au] OR “Cangelosi G”[au] OR “Escudero J”[au] OR “Cherkos A”[au] OR “Black D”[au] OR “Duncombe C”[au] OR “Barrett H”[au] OR “Grundner C”[au] OR “Frando A”[au] OR “Tram K”[au] OR “Andriesen J”[au] OR “Sharma M”[au] OR “Gern B”[au] OR “West E”[au] OR “Maciag K”[au] OR “Urdahl K”[au] OR “Wier J”[au] OR “Mayer-Blackwell K”[au] OR “Adams K”[au] OR “Pruneda A”[au] OR “Simmons J”[au] OR “Pepple K”[au] OR “Larsen Akins S”[au] OR “Ranade D”[au] OR “Allen R”[au] OR “Motiri F”[au] OR “Coler R”[au] OR “Chohan B”[au] OR “ Richardson S”[au] OR “Fredricks L”[au] OR “Thong Q”[au] OR “Krug S”[au] OR “Bhuiyan M”[au] OR “Akins S”[au] OR “Brumwell A”[au] OR “ King`ori B”[au] OR “Kearney J”[au] OR “Ross J”[au] OR “Johnson A”[au] OR “Layton E”[au] OR “Hernandez R”[au] OR “Himmelfarb T”[au] OR “Graham 

# 3) Submit the searches

An example showing the output we're parsing

In [9]:
url_test = base + '("Moosa Y"[au] OR "Hawn T"[au])' + '+AND+'+ '("University of KwaZulu-Natal"[ad] OR "University of Washington"[ad])' + '+AND+' + concept_final + '&usehistory=y'
url_test

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=2023/03/01:2023/06/30[pdat]("Moosa Y"[au] OR "Hawn T"[au])+AND+("University of KwaZulu-Natal"[ad] OR "University of Washington"[ad])+AND+("tubercul*"[tw] OR "Antitubercul*"[tw] OR "Anti-Tubercul*"[tw] OR "osteotubercul*"[tw] OR "nephrotubercul*"[tw] OR "anthracosilicotubercul*"[tw] OR "coniotubercul*"[tw] OR "Tuberculin"[tw] OR "tb"[tw] OR "xdr-tb"[tw] OR "xdrtb"[tw] OR "mdr-tb"[tw] OR "mdrtb"[tw] OR "phthisis"[tw] OR "pneumonophthisis"[tw] OR "pneumophthisiology"[tw] OR "silicotubercul*"[tw] OR "bazin disease"[tw] OR "erythema induratum"[tw] OR "white swelling"[tw] OR "king`s evil"[tw] OR "scrofula"[tw] OR "pott disease"[tw] OR "koch`s disease"[tw] OR "Interferon-gamma Release Test"[tw] OR "Tuberculosis"[mh] OR "Mycobacterium tuberculosis"[mh] OR "Antitubercular Agents"[mh] OR "Tuberculin Test"[mh] OR "Interferon-gamma Release Tests"[mh] OR "Tuberculosis Vaccines"[mh])&usehistory=y'

In [13]:
page = requests.get(url_test).text
result = BeautifulSoup(page, 'xml')
result

# 37058459

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">
<eSearchResult><Count>1202</Count><RetMax>20</RetMax><RetStart>0</RetStart><QueryKey>1</QueryKey><WebEnv>MCID_654aae7a0e5deb52bf3583d3</WebEnv><IdList>
<Id>37908762</Id>
<Id>37902822</Id>
<Id>37860976</Id>
<Id>37821085</Id>
<Id>37800573</Id>
<Id>37725440</Id>
<Id>37716189</Id>
<Id>37709959</Id>
<Id>37708378</Id>
<Id>37704889</Id>
<Id>37696514</Id>
<Id>37653382</Id>
<Id>37638056</Id>
<Id>37556423</Id>
<Id>37524010</Id>
<Id>37468353</Id>
<Id>37455004</Id>
<Id>37423324</Id>
<Id>37402589</Id>
<Id>37397728</Id>
</IdList><TranslationSet/><QueryTranslation>((2023/03/01:2023/06/30[Date - Publication] AND "Deshpande A"[Author]) OR "Johnson A"[Author] OR "Himmelfarb T"[Author] OR "John-Stewart G"[Author] OR "mayer blackwell k"[Author] OR "Wier J"[Author] OR "Tieu E"[Author] OR "Henry N"[Author] OR "Sorri Y"[Author] OR "Fajans 

In [93]:
page

''

Submit the search and get the results

In [48]:
page = requests.get(url).text
result = BeautifulSoup(page, 'xml')

Get the PubMed IDs (to be copy/pasted into PubMed)

In [88]:
IDlist = [i.text for i in result.find_all('Id')]
out = ';'.join(IDlist)

print(out)